In [1]:
# Cell 1: Import necessary libraries
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from IPython.display import display, Markdown
import warnings
import joblib
import os
import json

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Cell 2: Load Data with Proper Encoding
try:
    # Try different encodings to handle special characters
    encodings_to_try = ['utf-8', 'latin-1', 'cp1252', 'utf-8-sig']
    
    qualifying_df = None
    races_df = None
    
    # Load qualifying data
    for encoding in encodings_to_try:
        try:
            qualifying_df = pd.read_csv("../data/sprint_processed/updated_qualifying.csv", 
                                      parse_dates=["date"], encoding=encoding)
            print(f"Qualifying data loaded successfully with {encoding} encoding")
            break
        except UnicodeDecodeError:
            continue
    
    if qualifying_df is None:
        raise UnicodeDecodeError("Could not decode qualifying CSV with any standard encoding")
    
    # Load races data
    for encoding in encodings_to_try:
        try:
            races_df = pd.read_csv("../data/sprint_processed/updated_races.csv", 
                                 parse_dates=["date"], encoding=encoding)
            print(f"Races data loaded successfully with {encoding} encoding")
            break
        except UnicodeDecodeError:
            continue
    
    if races_df is None:
        raise UnicodeDecodeError("Could not decode races CSV with any standard encoding")
    
    print("CSV files loaded successfully.")
    print(f"Races data includes {races_df['season'].max()} season up to round {races_df[races_df['season'] == races_df['season'].max()]['round'].max()}")

except FileNotFoundError as e:
    print(f"Error: File not found. {e}")
    raise
except Exception as e:
    print(f"Error loading files: {e}")
    raise

# Basic data cleaning
print("Applying basic data cleaning...")
races_df['points'] = pd.to_numeric(races_df['points'], errors='coerce').fillna(0)
races_df['position'] = pd.to_numeric(races_df['position'], errors='coerce')
races_df['grid'] = pd.to_numeric(races_df['grid'], errors='coerce')

def clean_driver_name(name):
    if isinstance(name, str):
        name = name.replace(' Jr.', '').replace('Hülkenberg', 'Hulkenberg').replace('Perez', 'Pérez').replace('Raikkonen', 'Räikkönen')
        if "Antonelli" in name:
            return "Kimi Antonelli"
    return name

races_df['driverFullName'] = races_df['driverFullName'].apply(clean_driver_name)
qualifying_df['driverFullName'] = qualifying_df['driverFullName'].apply(clean_driver_name)

# Ensure IDs are strings
id_cols = ['driverId', 'constructorId']
for col in id_cols:
    if col in races_df.columns:
        races_df[col] = races_df[col].astype(str)
    if col in qualifying_df.columns:
        qualifying_df[col] = qualifying_df[col].astype(str)

print("Data cleaning complete.")

Qualifying data loaded successfully with utf-8 encoding
Races data loaded successfully with utf-8 encoding
CSV files loaded successfully.
Races data includes 2025.0 season up to round 19.0
Applying basic data cleaning...
Data cleaning complete.


In [3]:
# Cell 3: Feature Engineering and Model Setup
def calculate_features(df_races, df_qualifying):
    """Calculate rolling metrics and standings features"""
    print("Calculating features...")
    df_races_processed = df_races.sort_values(by=["season", "round", "date"]).copy()
    
    # Merge names if needed
    if 'driverFullName' not in df_races_processed.columns or 'constructorName' not in df_races_processed.columns:
        print("Merging names from qualifying data...")
        latest_qual_names = df_qualifying.sort_values(by="date", ascending=False).drop_duplicates(subset=["driverId", "constructorId"])
        name_map_df = latest_qual_names[["driverId", "constructorId", "driverFullName", "constructorName"]].copy()
        
        driver_id_to_name = name_map_df.drop_duplicates(subset="driverId").set_index("driverId")["driverFullName"]
        constructor_id_to_name = name_map_df.drop_duplicates(subset="constructorId").set_index("constructorId")["constructorName"]
        
        df_races_processed["driverFullName"] = df_races_processed["driverId"].map(driver_id_to_name)
        df_races_processed["constructorName"] = df_races_processed["constructorId"].map(constructor_id_to_name)
        
        df_races_processed["driverFullName"].fillna(df_races_processed["driverId"], inplace=True)
        df_races_processed["constructorName"].fillna(df_races_processed["constructorId"], inplace=True)
    
    # Target and features
    df_races_processed["is_winner"] = (df_races_processed["position"] == 1).astype(int)
    df_races_processed["grid"] = df_races_processed["grid"].replace(0, 21).fillna(21).astype(int)
    
    # Rolling performance metrics
    df_races_processed = df_races_processed.sort_values(by=["driverId", "season", "round", "date"])
    rolling_features_cols = ["points", "position", "grid"]
    
    for feature_col in rolling_features_cols:
        roll_mean = df_races_processed.groupby("driverId")[feature_col].rolling(window=5, min_periods=1).mean()
        df_races_processed[f"avg_{feature_col}_last_5"] = roll_mean.groupby(level=0).shift(1).reset_index(level=0, drop=True)
    
    # Championship standings
    df_races_processed["season_points"] = df_races_processed.groupby(["season", "driverId"])["points"].cumsum()
    df_races_processed["points_standings_prev_race"] = df_races_processed.groupby(["season", "driverId"])["season_points"].shift(1)
    
    # Fill NaNs
    df_races_processed["avg_points_last_5"].fillna(0, inplace=True)
    df_races_processed["avg_position_last_5"].fillna(21, inplace=True)
    df_races_processed["avg_grid_last_5"].fillna(21, inplace=True)
    df_races_processed["points_standings_prev_race"].fillna(0, inplace=True)
    
    df_races_processed = df_races_processed.drop(columns=["season_points"], errors='ignore')
    return df_races_processed.sort_values(by=["season", "round", "date"])

# Execute feature engineering
data_df_featured = calculate_features(races_df.copy(), qualifying_df.copy())

# Create name maps
latest_driver_entries = data_df_featured.drop_duplicates(subset="driverFullName", keep="last")
latest_driver_name_to_id_map = latest_driver_entries.set_index("driverFullName")["driverId"].to_dict()

latest_constructor_entries = data_df_featured.drop_duplicates(subset="constructorName", keep="last")
latest_constructor_name_to_id_map = latest_constructor_entries.set_index("constructorName")["constructorId"].to_dict()

Calculating features...


In [4]:
# Cell 4: Model Training and Saving (consolidated)
# Define features
features = [
    "grid", "circuitId", "driverId", "constructorId",
    "avg_points_last_5", "avg_position_last_5", "avg_grid_last_5", "points_standings_prev_race"
]
target = "is_winner"

numerical_features = ["grid", "avg_points_last_5", "avg_position_last_5", "avg_grid_last_5", "points_standings_prev_race"]
categorical_features = ["circuitId", "driverId", "constructorId"]

# Create preprocessing pipelines
numerical_transformer = SimpleImputer(strategy="median")
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numerical_transformer, numerical_features),
    ("cat", categorical_transformer, categorical_features)
])

# Create and train model
model_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", GradientBoostingClassifier(
        n_estimators=100, learning_rate=0.1, max_depth=3, subsample=0.8, random_state=42
    ))
])

X = data_df_featured[features]
y = data_df_featured[target]

print("Training model...")
model_pipeline.fit(X, y)

# Save model
model_filename = "joblogs/f1_winner_predictor_model_gbc_sprint.joblib"
os.makedirs("joblogs", exist_ok=True)
joblib.dump(model_pipeline, model_filename)
print(f"Model saved to {model_filename}")

Training model...
Model saved to joblogs/f1_winner_predictor_model_gbc_sprint.joblib


In [5]:
# Cell 5: Prediction Functions (consolidated)
TEAM_REBRAND_MAP = {
    
    # Red Bull rebrands
    "Red Bull Racing Honda RBPT": "red_bull",
    
    # McLaren rebrands
    "McLaren Mercedes": "mclaren", 
    "McLaren-Mercedes": "mclaren",
    
    # Ferrari rebrands
    "Ferrari": "ferrari", 
    
    # Mercedes rebrands
    "Mercedes": "mercedes",
    
    # Racing Bull rebrands
    "Racing Bulls Honda RBPT": "rb", 
    "Racing Bulls-Honda RBPT": "rb",
    "AlphaTauri": "alphatauri",
    "RB F1 Team": "rb",
    
    # Williams rebrands
    "Williams Mercedes": "williams", 
    "Williams-Mercedes": "williams",
    
    # Haas rebrands
    "Haas Ferrari": "haas", 
    "Haas-Ferrari": "haas",
    
    # Alpine rebrands
    "Alpine Renault": "alpine", 
    "Alpine-Renault": "alpine",
    "Renault": "renault",
    
    # Aston Martin rebrands
    "Aston Martin Aramco Mercedes": "aston_martin", 
    "Aston Martin Aramco-Mercedes": "aston_martin",
    "Racing Point": "racing_point",
    
    # Sauber rebrands
    "Kick Sauber Ferrari": "sauber", 
    "Kick Sauber-Ferrari": "sauber",
    "Alfa Romeo": "alfa",
    "Sauber": "sauber"
}

def prepare_grid_for_prediction(grid, driver_name_to_id_map, constructor_name_to_id_map, team_rebrand_map):
    """Prepare the race grid for prediction by mapping names to IDs while preserving display names."""
    prepared_grid = []
    
    for entry in grid:
        driver_name = clean_driver_name(entry['driver'])
        team_name = entry['team']  # Original full name like "Williams-Mercedes"
        
        # Map team name to constructor ID for the model
        constructor_id = None
        for original, rebrand_id in team_rebrand_map.items():
            if original in team_name:
                constructor_id = rebrand_id
                break
        
        # Fallback if no match found
        if not constructor_id:
            constructor_id = team_name.lower().replace(' ', '_').replace('-', '_')
        
        # Get driver ID
        driver_id = driver_name_to_id_map.get(
            driver_name, 
            f"unknown_{driver_name.lower().replace(' ', '_')}"
        )
        
        prepared_grid.append({
            'driverId': driver_id,
            'driverFullName': driver_name,
            'constructorId': constructor_id,
            'constructorName': team_name,  # ✅ Keep original full team name
            'grid': entry['grid']
        })
    
    return prepared_grid

def predict_race_winner_probabilities(
    circuit_id, predict_rows_featured, model, model_features_list, driver_detail_map
):
    """Predict win probabilities for drivers and merge duplicates"""
    if predict_rows_featured.empty or not model or not driver_detail_map:
        return {}

    try:
        # Run model prediction
        predict_X = predict_rows_featured[model_features_list]
        probabilities = model.predict_proba(predict_X)
        win_probabilities = probabilities[:, 1]

        # Normalize probabilities
        total_prob = np.sum(win_probabilities)
        normalized_probs = (
            win_probabilities / total_prob
            if total_prob > 0
            else np.ones(len(predict_X)) / len(predict_X)
        )

        # Collect results
        results = {}
        for i, index in enumerate(predict_rows_featured.index):
            model_driver_id = predict_rows_featured.loc[index, "driverId"]
            details = driver_detail_map.get(model_driver_id, {})

            # Always prefer FullName from driver_detail_map
            display_name = details.get("FullName", model_driver_id)

            results[model_driver_id] = {
                "Probability": normalized_probs[i],
                "DriverFullName": display_name,
                "ConstructorName": details.get("ConstructorName", "Unknown"),  # ✅ Now has full name
                "Grid": details.get("Grid", predict_rows_featured.loc[index, "grid"]),
            }

        # Merge duplicates by normalized display name
        merged_results = {}
        for driver_id, details in results.items():
            normalized_key = details["DriverFullName"].lower().replace(" ", "_")

            if normalized_key in merged_results:
                # Add probabilities if duplicate
                merged_results[normalized_key]["Probability"] += details["Probability"]
            else:
                merged_results[normalized_key] = {
                    "Probability": details["Probability"],
                    "DriverFullName": details["DriverFullName"],
                    "ConstructorName": details["ConstructorName"],  # ✅ Full team name
                    "Grid": details["Grid"],
                }

        # Sort by probability
        return dict(
            sorted(
                merged_results.items(),
                key=lambda item: item[1]["Probability"],
                reverse=True,
            )
        )

    except Exception as e:
        print(f"Prediction error: {e}")
        return {}

In [6]:
# Cell 6: Race Configurations and Batch Prediction (Fixed)
RACE_CONFIGS = {
    "australian_gp": {
        "season": 2025, "round": 1, "circuit_id": "albert_park", "description": "2025 Australian Grand Prix",
        "grid": [
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 2},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 3},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 4},
            {'driver': 'Yuki Tsunoda', 'team': 'Racing Bulls Honda RBPT', 'grid': 5},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 6},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 7},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 8},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 9},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 10},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 11},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 12},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 13},
            {'driver': 'Jack Doohan', 'team': 'Alpine Renault', 'grid': 14},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 15},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 16},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 17},
            {'driver': 'Liam Lawson', 'team': 'Red Bull Racing Honda RBPT', 'grid': 18},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 19},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 20}
        ]
    },
    "chinese_gp": {
        "season": 2025, "round": 2, "circuit_id": "shanghai", "description": "2025 Chinese Grand Prix",
        "grid": [
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 2},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 3},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 4},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 5},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 6},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 7},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 8},
            {'driver': 'Yuki Tsunoda', 'team': 'Racing Bulls Honda RBPT', 'grid': 9},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 10},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 11},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 12},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 13},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 14},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 15},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 16},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 17},
            {'driver': 'Jack Doohan', 'team': 'Alpine Renault', 'grid': 18},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 19},
            {'driver': 'Liam Lawson', 'team': 'Red Bull Racing Honda RBPT', 'grid': 20}
        ]
    },
    "japanese_gp": {
        "season": 2025, "round": 3, "circuit_id": "suzuka", "description": "2025 Japanese Grand Prix",
        "grid": [
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 1},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 2},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 3},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 4},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 5},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 6},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 7},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 8},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 9},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 10},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 11},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 12},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 13},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 14},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 15},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 16},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 17},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 18},
            {'driver': 'Jack Doohan', 'team': 'Alpine Renault', 'grid': 19},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 20}
        ]
    },
    "bahrain_gp": {
        "season": 2025, "round": 4, "circuit_id": "bahrain", "description": "2025 Bahrain Grand Prix",
        "grid": [
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 2},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 3},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 4},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 5},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 6},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 7},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 8},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 9},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 10},
            {'driver': 'Jack Doohan', 'team': 'Alpine Renault', 'grid': 11},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 12},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 13},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 14},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 15},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 16},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 17},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 18},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 19},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 20}
        ]
    },
    "saudi_arabian_gp": {
        "season": 2025, "round": 5, "circuit_id": "jeddah", "description": "2025 Saudi Arabian Grand Prix",
        "grid": [
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 1},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 2},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 3},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 4},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 5},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 6},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 7},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 8},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 9},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 10},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 11},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 12},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 13},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 14},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 15},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 16},
            {'driver': 'Jack Doohan', 'team': 'Alpine Renault', 'grid': 17},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 18},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 19},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 20}
        ]
    },
    "miami_gp": {
        "season": 2025, "round": 6, "circuit_id": "miami", "description": "2025 Miami Grand Prix",
        "grid": [
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 1},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 2},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 3},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 4},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 5},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 6},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 7},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 8},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 9},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 10},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 11},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 12},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 13},
            {'driver': 'Jack Doohan', 'team': 'Alpine Renault', 'grid': 14},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 15},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 16},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 17},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 18},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 19},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 20}
        ]
    },
    "emilia_romagna_gp": {
        "season": 2025, "round": 7, "circuit_id": "imola", "description": "2025 Emilia Romagna Grand Prix",
        "grid": [
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 2},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 3},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 4},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 5},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 6},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 7},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 8},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 9},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 10},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 11},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 12},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 13},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 14},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 15},
            {'driver': 'Franco Colapinto', 'team': 'Alpine Renault', 'grid': 16},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 17},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 18},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 19},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 20}
        ]
    },
    "monaco_gp": {
        "season": 2025, "round": 8, "circuit_id": "monaco", "description": "2025 Monaco Grand Prix",
        "grid": [
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 2},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 3},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 4},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 5},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 6},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 7},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 8},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 9},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 10},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 11},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 12},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 13},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 14},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 15},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 16},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 17},
            {'driver': 'Franco Colapinto', 'team': 'Alpine Renault', 'grid': 18},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 19},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 20}
        ]
    },
    "spanish_gp": {
        "season": 2025, "round": 9, "circuit_id": "catalunya", "description": "2025 Spanish Grand Prix",
        "grid": [
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 2},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 3},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 4},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 5},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 6},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 7},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 8},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 9},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 10},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 11},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 12},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 13},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 14},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 15},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 16},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 17},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 18},
            {'driver': 'Franco Colapinto', 'team': 'Alpine Renault', 'grid': 19},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 20}
        ]
    },
    "canadian_gp": {
        "season": 2025, "round": 10, "circuit_id": "villeneuve", "description": "2025 Canadian Grand Prix",
        "grid": [
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 1},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 2},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 3},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 4},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 5},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 6},
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 7},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 8},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 9},
            {'driver': 'Franco Colapinto', 'team': 'Alpine Renault', 'grid': 10},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 11},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 12},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 13},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 14},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 15},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 16},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 17},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 18},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 19},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 20}
        ]
    },
    "austrian_gp": {
        "season": 2025, "round": 11, "circuit_id": "red_bull_ring", "description": "2025 Austrian Grand Prix",
        "grid": [
            {'driver': 'Lando Norris', 'team': 'McLaren Mercedes', 'grid': 1},
            {'driver': 'Charles Leclerc', 'team': 'Ferrari', 'grid': 2},
            {'driver': 'Oscar Piastri', 'team': 'McLaren Mercedes', 'grid': 3},
            {'driver': 'Lewis Hamilton', 'team': 'Ferrari', 'grid': 4},
            {'driver': 'George Russell', 'team': 'Mercedes', 'grid': 5},
            {'driver': 'Liam Lawson', 'team': 'Racing Bulls Honda RBPT', 'grid': 6},
            {'driver': 'Max Verstappen', 'team': 'Red Bull Racing Honda RBPT', 'grid': 7},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari', 'grid': 8},
            {'driver': 'Kimi Antonelli', 'team': 'Mercedes', 'grid': 9},
            {'driver': 'Pierre Gasly', 'team': 'Alpine Renault', 'grid': 10},
            {'driver': 'Fernando Alonso', 'team': 'Aston Martin Aramco Mercedes', 'grid': 11},
            {'driver': 'Alexander Albon', 'team': 'Williams Mercedes', 'grid': 12},
            {'driver': 'Isack Hadjar', 'team': 'Racing Bulls Honda RBPT', 'grid': 13},
            {'driver': 'Franco Colapinto', 'team': 'Alpine Renault', 'grid': 14},
            {'driver': 'Oliver Bearman', 'team': 'Haas Ferrari', 'grid': 15},
            {'driver': 'Lance Stroll', 'team': 'Aston Martin Aramco Mercedes', 'grid': 16},
            {'driver': 'Esteban Ocon', 'team': 'Haas Ferrari', 'grid': 17},
            {'driver': 'Yuki Tsunoda', 'team': 'Red Bull Racing Honda RBPT', 'grid': 18},
            {'driver': 'Carlos Sainz', 'team': 'Williams Mercedes', 'grid': 19},
            {'driver': 'Nico Hulkenberg', 'team': 'Kick Sauber Ferrari', 'grid': 20}
        ]
    },
    "british_gp": {
        "season": 2025, "round": 12, "circuit_id": "silverstone", "description": "2025 British Grand Prix",
        "grid": [
            {'driver': 'Lando Norris',      'team': 'McLaren Mercedes',              'grid': 3},
            {'driver': 'Charles Leclerc',   'team': 'Ferrari',                       'grid': 6},
            {'driver': 'Oscar Piastri',     'team': 'McLaren Mercedes',              'grid': 2},
            {'driver': 'Lewis Hamilton',    'team': 'Ferrari',                       'grid': 5},
            {'driver': 'George Russell',    'team': 'Mercedes',                      'grid': 4},
            {'driver': 'Liam Lawson',       'team': 'Racing Bulls Honda RBPT',       'grid': 16},
            {'driver': 'Max Verstappen',    'team': 'Red Bull Racing Honda RBPT',    'grid': 1},
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari',           'grid': 17},
            {'driver': 'Kimi Antonelli',    'team': 'Mercedes',                      'grid': 7},
            {'driver': 'Pierre Gasly',      'team': 'Alpine Renault',                'grid': 10},
            {'driver': 'Fernando Alonso',   'team': 'Aston Martin Aramco Mercedes',  'grid': 9},
            {'driver': 'Alexander Albon',   'team': 'Williams Mercedes',             'grid': 14},
            {'driver': 'Isack Hadjar',      'team': 'Racing Bulls Honda RBPT',       'grid': 13},
            {'driver': 'Franco Colapinto',  'team': 'Alpine Renault',                'grid': 20},
            {'driver': 'Oliver Bearman',    'team': 'Haas Ferrari',                  'grid': 8},
            {'driver': 'Lance Stroll',      'team': 'Aston Martin Aramco Mercedes',  'grid': 18},
            {'driver': 'Esteban Ocon',      'team': 'Haas Ferrari',                  'grid': 15},
            {'driver': 'Yuki Tsunoda',      'team': 'Red Bull Racing Honda RBPT',    'grid': 12},
            {'driver': 'Carlos Sainz',      'team': 'Williams Mercedes',             'grid': 11},
            {'driver': 'Nico Hulkenberg',   'team': 'Kick Sauber Ferrari',           'grid': 19}
        ]
    },
    "spa_gp": {
        "season": 2025, "round": 13, "circuit_id": "spa", "description": "2025 Belgian Grand Prix",
        "grid": [
            {'driver': 'Lando Norris',      'team': 'McLaren Mercedes',              'grid': 1},
            {'driver': 'Oscar Piastri',     'team': 'McLaren Mercedes',              'grid': 2},
            
            {'driver': 'Charles Leclerc',   'team': 'Ferrari',                       'grid': 3},
            {'driver': 'Lewis Hamilton',    'team': 'Ferrari',                       'grid': 16},
            
            {'driver': 'George Russell',    'team': 'Mercedes',                      'grid': 6},
            {'driver': 'Kimi Antonelli',    'team': 'Mercedes',                      'grid': 18},
            
            {'driver': 'Max Verstappen',    'team': 'Red Bull Racing Honda RBPT',    'grid': 4},
            {'driver': 'Yuki Tsunoda',      'team': 'Red Bull Racing Honda RBPT',    'grid': 7},
            
            {'driver': 'Fernando Alonso',   'team': 'Aston Martin Aramco Mercedes',  'grid': 19},
            {'driver': 'Lance Stroll',      'team': 'Aston Martin Aramco Mercedes',  'grid': 20},
            
            {'driver': 'Alexander Albon',   'team': 'Williams Mercedes',             'grid': 5},
            {'driver': 'Carlos Sainz',      'team': 'Williams Mercedes',             'grid': 15},
            
            {'driver': 'Isack Hadjar',      'team': 'Racing Bulls Honda RBPT',       'grid': 8},
            {'driver': 'Liam Lawson',       'team': 'Racing Bulls Honda RBPT',       'grid': 9},
            
            {'driver': 'Pierre Gasly',      'team': 'Alpine Renault',                'grid': 13},
            {'driver': 'Franco Colapinto',  'team': 'Alpine Renault',                'grid': 17},
            
            {'driver': 'Oliver Bearman',    'team': 'Haas Ferrari',                  'grid': 12},
            {'driver': 'Esteban Ocon',      'team': 'Haas Ferrari',                  'grid': 13},
            
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari',           'grid': 10},
            {'driver': 'Nico Hulkenberg',   'team': 'Kick Sauber Ferrari',           'grid': 14}
        ]
    },
    "hungarian_gp": {
        "season": 2025, "round": 14, "circuit_id": "hungaroring", "description": "2025 Hungarian Grand Prix",
        "grid": [
            {'driver': 'Lando Norris',      'team': 'McLaren Mercedes',              'grid': 3},
            {'driver': 'Oscar Piastri',     'team': 'McLaren Mercedes',              'grid': 2},
            
            {'driver': 'Charles Leclerc',   'team': 'Ferrari',                       'grid': 1},
            {'driver': 'Lewis Hamilton',    'team': 'Ferrari',                       'grid': 12},
            
            {'driver': 'George Russell',    'team': 'Mercedes',                      'grid': 4},
            {'driver': 'Kimi Antonelli',    'team': 'Mercedes',                      'grid': 15},
            
            {'driver': 'Max Verstappen',    'team': 'Red Bull Racing Honda RBPT',    'grid': 8},
            {'driver': 'Yuki Tsunoda',      'team': 'Red Bull Racing Honda RBPT',    'grid': 16},
            
            {'driver': 'Fernando Alonso',   'team': 'Aston Martin Aramco Mercedes',  'grid': 5},
            {'driver': 'Lance Stroll',      'team': 'Aston Martin Aramco Mercedes',  'grid': 6},
            
            {'driver': 'Alexander Albon',   'team': 'Williams Mercedes',             'grid': 20},
            {'driver': 'Carlos Sainz',      'team': 'Williams Mercedes',             'grid': 13},
            
            {'driver': 'Isack Hadjar',      'team': 'Racing Bulls Honda RBPT',       'grid': 10},
            {'driver': 'Liam Lawson',       'team': 'Racing Bulls Honda RBPT',       'grid': 9},
            
            {'driver': 'Pierre Gasly',      'team': 'Alpine Renault',                'grid': 17},
            {'driver': 'Franco Colapinto',  'team': 'Alpine Renault',                'grid': 14},
            
            {'driver': 'Oliver Bearman',    'team': 'Haas Ferrari',                  'grid': 11},
            {'driver': 'Esteban Ocon',      'team': 'Haas Ferrari',                  'grid': 18},
            
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari',           'grid': 7},
            {'driver': 'Nico Hulkenberg',   'team': 'Kick Sauber Ferrari',           'grid': 19}
        ]
    },
    "dutch_gp": {
        "season": 2025, "round": 15, "circuit_id": "zandvoort", "description": "2025 Dutch Grand Prix",
        "grid": [
            {'driver': 'Lando Norris',      'team': 'McLaren Mercedes',              'grid': 2},
            {'driver': 'Oscar Piastri',     'team': 'McLaren Mercedes',              'grid': 1},
            
            {'driver': 'Charles Leclerc',   'team': 'Ferrari',                       'grid': 6},
            {'driver': 'Lewis Hamilton',    'team': 'Ferrari',                       'grid': 7},
            
            {'driver': 'George Russell',    'team': 'Mercedes',                      'grid': 5},
            {'driver': 'Kimi Antonelli',    'team': 'Mercedes',                      'grid': 11},
            
            {'driver': 'Max Verstappen',    'team': 'Red Bull Racing Honda RBPT',    'grid': 3},
            {'driver': 'Yuki Tsunoda',      'team': 'Red Bull Racing Honda RBPT',    'grid': 12},
            
            {'driver': 'Fernando Alonso',   'team': 'Aston Martin Aramco Mercedes',  'grid': 10},
            {'driver': 'Lance Stroll',      'team': 'Aston Martin Aramco Mercedes',  'grid': 20},
            
            {'driver': 'Alexander Albon',   'team': 'Williams Mercedes',             'grid': 15},
            {'driver': 'Carlos Sainz',      'team': 'Williams Mercedes',             'grid': 9},
            
            {'driver': 'Isack Hadjar',      'team': 'Racing Bulls Honda RBPT',       'grid': 4},
            {'driver': 'Liam Lawson',       'team': 'Racing Bulls Honda RBPT',       'grid': 8},
            
            {'driver': 'Pierre Gasly',      'team': 'Alpine Renault',                'grid': 14},
            {'driver': 'Franco Colapinto',  'team': 'Alpine Renault',                'grid': 16},
            
            {'driver': 'Oliver Bearman',    'team': 'Haas Ferrari',                  'grid': 19},
            {'driver': 'Esteban Ocon',      'team': 'Haas Ferrari',                  'grid': 18},
            
            {'driver': 'Gabriel Bortoleto', 'team': 'Kick Sauber Ferrari',           'grid': 13},
            {'driver': 'Nico Hulkenberg',   'team': 'Kick Sauber Ferrari',           'grid': 17}
        ]
    },
    "italian_gp": {
        "season": 2025, "round": 16, "circuit_id": "monza", "description": "2025 Italian Grand Prix",
        "grid": [
        
            {"driver": "Lando Norris",      "team": "McLaren Mercedes",               "grid": 2},
            {"driver": "Oscar Piastri",     "team": "McLaren Mercedes",               "grid": 3},

            {"driver": "Charles Leclerc",   "team": "Ferrari",                        "grid": 4},
            {"driver": "Lewis Hamilton",    "team": "Ferrari",                        "grid": 5},

            {"driver": "George Russell",    "team": "Mercedes",                       "grid": 6},
            {"driver": "Andrea Kimi Antonelli", "team": "Mercedes",                   "grid": 7},

            {"driver": "Lance Stroll",      "team": "Aston Martin Aramco Mercedes",   "grid": 17},
            {"driver": "Fernando Alonso",   "team": "Aston Martin Aramco Mercedes",   "grid": 9},

            {"driver": "Max Verstappen",    "team": "Red Bull Racing Honda RBPT",     "grid": 1},
            {"driver": "Yuki Tsunoda",      "team": "Red Bull Racing Honda RBPT",     "grid": 10},

            {"driver": "Oliver Bearman",    "team": "Haas Ferrari",                   "grid": 11},
            {"driver": "Esteban Ocon",      "team": "Haas Ferrari",                   "grid": 15},

            {"driver": "Gabriel Bortoleto", "team": "Kick Sauber Ferrari",            "grid": 8},
            {"driver": "Nico Hulkenberg",   "team": "Kick Sauber Ferrari",            "grid": 12},

            {"driver": "Carlos Sainz",      "team": "Williams Mercedes",              "grid": 13},
            {"driver": "Alexander Albon",   "team": "Williams Mercedes",              "grid": 14},

            {"driver": "Franco Colapinto",  "team": "Alpine Renault",                 "grid": 18},
            {"driver": "Pierre Gasly",      "team": "Alpine Renault",                 "grid": 19},

            {"driver": "Isack Hadjar",      "team": "Racing Bulls Honda RBPT",        "grid": 16},
            {"driver": "Liam Lawson",       "team": "Racing Bulls Honda RBPT",        "grid": 20}
        ]
    },
    "baku_gp": {
        "season": 2025, "round": 17, "circuit_id": "baku", "description": "2025 Azerbaijan Grand Prix",
        "grid": [
        
            {"driver": "Lando Norris",      "team": "McLaren Mercedes",               "grid": 7},
            {"driver": "Oscar Piastri",     "team": "McLaren Mercedes",               "grid": 9},

            {"driver": "Charles Leclerc",   "team": "Ferrari",                        "grid": 10},
            {"driver": "Lewis Hamilton",    "team": "Ferrari",                        "grid": 12},

            {"driver": "George Russell",    "team": "Mercedes",                       "grid": 5},
            {"driver": "Andrea Kimi Antonelli", "team": "Mercedes",                   "grid": 4},

            {"driver": "Lance Stroll",      "team": "Aston Martin Aramco Mercedes",   "grid": 14},
            {"driver": "Fernando Alonso",   "team": "Aston Martin Aramco Mercedes",   "grid": 11},

            {"driver": "Max Verstappen",    "team": "Red Bull Racing Honda RBPT",     "grid": 1},
            {"driver": "Yuki Tsunoda",      "team": "Red Bull Racing Honda RBPT",     "grid": 6},

            {"driver": "Oliver Bearman",    "team": "Haas Ferrari",                   "grid": 15},
            {"driver": "Esteban Ocon",      "team": "Haas Ferrari",                   "grid": 20},

            {"driver": "Gabriel Bortoleto", "team": "Kick Sauber Ferrari",            "grid": 13},
            {"driver": "Nico Hulkenberg",   "team": "Kick Sauber Ferrari",            "grid": 17},

            {"driver": "Carlos Sainz",      "team": "Williams Mercedes",              "grid": 2},
            {"driver": "Alexander Albon",   "team": "Williams Mercedes",              "grid": 19},

            {"driver": "Franco Colapinto",  "team": "Alpine Renault",                 "grid": 16},
            {"driver": "Pierre Gasly",      "team": "Alpine Renault",                 "grid": 18},

            {"driver": "Isack Hadjar",      "team": "Racing Bulls Honda RBPT",        "grid": 8},
            {"driver": "Liam Lawson",       "team": "Racing Bulls Honda RBPT",        "grid": 3}
        ]
    },
    "marina_bay_gp": {
        "season": 2025, "round": 18, "circuit_id": "marina_bay", "description": "2025 Singapore Grand Prix",
        "grid": [
            {"driver": "George Russell",       "team": "Mercedes",        "grid": 1},
            {"driver": "Kimi Antonelli",       "team": "Mercedes",        "grid": 4},

            {"driver": "Oscar Piastri",        "team": "McLaren Mercedes","grid": 3},
            {"driver": "Lando Norris",         "team": "McLaren Mercedes","grid": 5},
        
            {"driver": "Lewis Hamilton",       "team": "Ferrari",         "grid": 6},
            {"driver": "Charles Leclerc",      "team": "Ferrari",         "grid": 7},
            
            {"driver": "Max Verstappen",       "team": "Red Bull Racing Honda RBPT", "grid": 2},
            {"driver": "Yuki Tsunoda",         "team": "Red Bull Racing Honda RBPT", "grid": 13},
        
            {"driver": "Isack Hadjar",         "team": "Racing Bulls Honda RBPT", "grid": 8},
            {"driver": "Liam Lawson",          "team": "Racing Bulls Honda RBPT", "grid": 12},
        
            {"driver": "Fernando Alonso",      "team": "Aston Martin Aramco Mercedes", "grid": 10},
            {"driver": "Lance Stroll",         "team": "Aston Martin Aramco Mercedes", "grid": 15},
    
            {"driver": "Nico Hulkenberg",      "team": "Kick Sauber Ferrari", "grid": 11},
            {"driver": "Gabriel Bortoleto",    "team": "Kick Sauber Ferrari", "grid": 14},
            
            {"driver": "Oliver Bearman",       "team": "Haas Ferrari", "grid": 9},
            {"driver": "Esteban Ocon",         "team": "Haas Ferrari", "grid": 17},
        
            {"driver": "Carlos Sainz",         "team": "Williams Mercedes", "grid": 18},
            {"driver": "Alexander Albon",      "team": "Williams Mercedes", "grid": 19},
        
            {"driver": "Franco Colapinto",     "team": "Alpine Renault", "grid": 16},
            {"driver": "Pierre Gasly",         "team": "Alpine Renault", "grid": 20}
        ]
    },
    "americas_gp": {
        "season": 2025, "round": 19, "circuit_id": "americas", "description": "2025 United States Grand Prix",
        "grid": [
            
            {"driver": "George Russell",      "team": "Mercedes",        "grid": 4},
            {"driver": "Kimi Antonelli",      "team": "Mercedes",        "grid": 7},

            
            {"driver": "Oscar Piastri",       "team": "McLaren Mercedes","grid": 6},
            {"driver": "Lando Norris",        "team": "McLaren Mercedes","grid": 2},

            
            {"driver": "Lewis Hamilton",      "team": "Ferrari",         "grid": 5},
            {"driver": "Charles Leclerc",     "team": "Ferrari",         "grid": 3},

            
            {"driver": "Max Verstappen",      "team": "Red Bull Racing Honda RBPT", "grid": 1},
            {"driver": "Yuki Tsunoda",        "team": "Red Bull Racing Honda RBPT", "grid": 13},

            
            {"driver": "Isack Hadjar",        "team": "Racing Bulls Honda RBPT", "grid": 19},
            {"driver": "Liam Lawson",         "team": "Racing Bulls Honda RBPT", "grid": 12},

            
            {"driver": "Fernando Alonso",     "team": "Aston Martin Aramco Mercedes", "grid": 10},
            {"driver": "Lance Stroll",        "team": "Aston Martin Aramco Mercedes", "grid": 20},

            
            {"driver": "Nico Hulkenberg",     "team": "Kick Sauber Ferrari", "grid": 11},
            {"driver": "Gabriel Bortoleto",   "team": "Kick Sauber Ferrari", "grid": 16},

            
            {"driver": "Oliver Bearman",      "team": "Haas Ferrari", "grid": 8},
            {"driver": "Esteban Ocon",        "team": "Haas Ferrari", "grid": 17},

            
            {"driver": "Carlos Sainz",        "team": "Williams Mercedes", "grid": 9},
            {"driver": "Alexander Albon",     "team": "Williams Mercedes", "grid": 18},

            
            {"driver": "Franco Colapinto",    "team": "Alpine Renault", "grid": 15},
            {"driver": "Pierre Gasly",        "team": "Alpine Renault", "grid": 14}
        ]
    },
    "mexico_gp": {
    "season": 2025, "round": 20, "circuit_id": "rodriguez", "description": "2025 Mexican Grand Prix",
    "grid": [
    
            {"driver": "George Russell",      "team": "Mercedes",        "grid": 4},
            {"driver": "Kimi Antonelli",      "team": "Mercedes",        "grid": 6},

            
            {"driver": "Oscar Piastri",       "team": "McLaren Mercedes","grid": 8},
            {"driver": "Lando Norris",        "team": "McLaren Mercedes","grid": 1},

            
            {"driver": "Lewis Hamilton",      "team": "Ferrari",         "grid": 3},
            {"driver": "Charles Leclerc",     "team": "Ferrari",         "grid": 2},

            
            {"driver": "Max Verstappen",      "team": "Red Bull Racing Honda RBPT", "grid": 5},
            {"driver": "Yuki Tsunoda",        "team": "Red Bull Racing Honda RBPT", "grid": 11},

            
            {"driver": "Isack Hadjar",        "team": "Racing Bulls Honda RBPT", "grid": 9},
            {"driver": "Liam Lawson",         "team": "Racing Bulls Honda RBPT", "grid": 15},

            
            {"driver": "Fernando Alonso",     "team": "Aston Martin Aramco Mercedes", "grid": 14},
            {"driver": "Lance Stroll",        "team": "Aston Martin Aramco Mercedes", "grid": 19},

            
            {"driver": "Nico Hulkenberg",     "team": "Kick Sauber Ferrari", "grid": 13},
            {"driver": "Gabriel Bortoleto",   "team": "Kick Sauber Ferrari", "grid": 16},

            
            {"driver": "Oliver Bearman",      "team": "Haas Ferrari", "grid": 10},
            {"driver": "Esteban Ocon",        "team": "Haas Ferrari", "grid": 12},

            
            {"driver": "Carlos Sainz",        "team": "Williams Mercedes", "grid": 7},
            {"driver": "Alexander Albon",     "team": "Williams Mercedes", "grid": 17},

        
            {"driver": "Franco Colapinto",    "team": "Alpine Renault", "grid": 20},
            {"driver": "Pierre Gasly",        "team": "Alpine Renault", "grid": 18}
        ]
    }

}

def predict_single_race(
    race_config,
    model,
    base_races_df,
    base_qualifying_df,
    driver_name_to_id_hist_map,
    constructor_name_to_id_hist_map,
    team_rebrand_map_current,
    model_features_list,
    save_path="predictions_GBC_Sprint",
):
    """Predict a single race with duplicate driver merge fix"""
    print(f"--- Predicting: {race_config['description']} ---")

    # Prepare grid
    prepared_grid = prepare_grid_for_prediction(
        race_config["grid"],
        driver_name_to_id_hist_map,
        constructor_name_to_id_hist_map,
        team_rebrand_map_current,
    )

    # Create driver detail map
    driver_detail_map = {
        item["driverId"]: {
            "FullName": item["driverFullName"],
            "ConstructorName": item["constructorName"],
            "Grid": item["grid"],
        }
        for item in prepared_grid
    }

    # Create future race DataFrame
    future_race_df = pd.DataFrame(prepared_grid)[
        ["driverId", "constructorId", "grid"]
    ].copy()

    # Assign columns properly
    future_race_df["season"] = race_config["season"]
    future_race_df["round"] = race_config["round"]
    future_race_df["circuitId"] = race_config["circuit_id"]
    future_race_df["date"] = pd.Timestamp.now()
    future_race_df["position"] = np.nan
    future_race_df["points"] = 0.0

    # Combine with historical data
    required_cols = [
        "season",
        "round",
        "date",
        "driverId",
        "constructorId",
        "circuitId",
        "grid",
        "position",
        "points",
    ]
    combined_df = pd.concat(
        [base_races_df[required_cols], future_race_df], ignore_index=True
    )

    # Calculate features
    combined_featured_df = calculate_features(combined_df, base_qualifying_df)

    # Get prediction rows
    predict_rows = combined_featured_df[
        (combined_featured_df["season"] == race_config["season"])
        & (combined_featured_df["round"] == race_config["round"])
    ].copy()

    if predict_rows.empty:
        print("Error: No prediction rows found")
        return

    # Predict (with duplicate merge fix)
    winner_probs = predict_race_winner_probabilities(
        race_config["circuit_id"],
        predict_rows,
        model,
        model_features_list,
        driver_detail_map,
    )

    if winner_probs:
        # Create results DataFrame
        results_df = pd.DataFrame(
            [
                {
                    "Driver": details["DriverFullName"],
                    "Grid": details["Grid"],
                    "Team": details["ConstructorName"],
                    "Probability": details["Probability"],
                }
                for _, details in winner_probs.items()
            ]
        )

        # Save to CSV
        if save_path:
            os.makedirs(save_path, exist_ok=True)
            safe_filename = "".join(
                c if c.isalnum() else "_" for c in race_config["description"]
            )
            csv_filename = os.path.join(
                save_path,
                f"{race_config['season']}_R{race_config['round']:02d}_{safe_filename}_{race_config['circuit_id']}_predictions_GBC_Sprint.csv",
            )
            results_df.to_csv(csv_filename, index=False, float_format="%.6f")
            print(f"Saved: {csv_filename}")

        # Display results
        display_df = results_df.copy()
        display_df["Probability"] = display_df["Probability"].map("{:.2%}".format)

        markdown_table = (
            "| Driver             | Grid | Team                           | Probability |\n"
        )
        markdown_table += (
            "|--------------------|------|--------------------------------|-------------|\n"
        )
        for _, row in display_df.iterrows():
            markdown_table += f"| {row['Driver']:<18} | {row['Grid']:<4} | {row['Team']:<30} | {row['Probability']:>11} |\n"

        display(Markdown(markdown_table))

def predict_all_races(
    race_configs,
    model_filename = "joblogs/f1_winner_predictor_model_gbc_sprint.joblib"
):
    """Predict all configured races using the fixed pipeline"""
    # Load model once
    try:
        model = joblib.load(model_filename)
        print(f"Model loaded from {model_filename}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Run predictions for all races
    for race_key, race_config in race_configs.items():
        predict_single_race(
            race_config,
            model,
            races_df,
            qualifying_df,
            latest_driver_name_to_id_map,
            latest_constructor_name_to_id_map,
            TEAM_REBRAND_MAP,
            features
        )
        print()  # Add spacing between races


def predict_specific_races(
    race_keys,
    race_configs=RACE_CONFIGS,
    model_filename = "joblogs/f1_winner_predictor_model_gbc_sprint.joblib"
):
    """Predict specific races by key using the fixed pipeline"""
    # Load model once
    try:
        model = joblib.load(model_filename)
        print(f"Model loaded from {model_filename}")
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    # Run predictions for selected races
    for race_key in race_keys:
        if race_key in race_configs:
            predict_single_race(
                race_configs[race_key],
                model,
                races_df,
                qualifying_df,
                latest_driver_name_to_id_map,
                latest_constructor_name_to_id_map,
                TEAM_REBRAND_MAP,
                features
            )
            print()  # Add spacing between races
        else:
            print(f"Warning: Race key '{race_key}' not found in configurations")
            
print("Race configurations loaded. Available races:")
for key, config in RACE_CONFIGS.items():
    print(f"  - {key}: {config['description']}")

Race configurations loaded. Available races:
  - australian_gp: 2025 Australian Grand Prix
  - chinese_gp: 2025 Chinese Grand Prix
  - japanese_gp: 2025 Japanese Grand Prix
  - bahrain_gp: 2025 Bahrain Grand Prix
  - saudi_arabian_gp: 2025 Saudi Arabian Grand Prix
  - miami_gp: 2025 Miami Grand Prix
  - emilia_romagna_gp: 2025 Emilia Romagna Grand Prix
  - monaco_gp: 2025 Monaco Grand Prix
  - spanish_gp: 2025 Spanish Grand Prix
  - canadian_gp: 2025 Canadian Grand Prix
  - austrian_gp: 2025 Austrian Grand Prix
  - british_gp: 2025 British Grand Prix
  - spa_gp: 2025 Belgian Grand Prix
  - hungarian_gp: 2025 Hungarian Grand Prix
  - dutch_gp: 2025 Dutch Grand Prix
  - italian_gp: 2025 Italian Grand Prix
  - baku_gp: 2025 Azerbaijan Grand Prix
  - marina_bay_gp: 2025 Singapore Grand Prix
  - americas_gp: 2025 United States Grand Prix
  - mexico_gp: 2025 Mexican Grand Prix


In [7]:
# Cell 7: Execute All Predictions
predict_all_races(RACE_CONFIGS)

Model loaded from joblogs/f1_winner_predictor_model_gbc_sprint.joblib
--- Predicting: 2025 Australian Grand Prix ---
Calculating features...
Merging names from qualifying data...
Saved: predictions_GBC_Sprint/2025_R01_2025_Australian_Grand_Prix_albert_park_predictions_GBC_Sprint.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Lando Norris       | 1    | McLaren Mercedes               |      34.30% |
| Oscar Piastri      | 2    | McLaren Mercedes               |       5.58% |
| Max Verstappen     | 3    | Red Bull Racing Honda RBPT     |       5.25% |
| George Russell     | 4    | Mercedes                       |       1.50% |
| Kimi Antonelli     | 16   | Mercedes                       |       0.18% |
| Charles Leclerc    | 7    | Ferrari                        |       0.17% |
| Lewis Hamilton     | 8    | Ferrari                        |       0.08% |
| Alexander Albon    | 6    | Williams Mercedes              |       0.07% |
| Fernando Alonso    | 12   | Aston Martin Aramco Mercedes   |       0.07% |
| Oliver Bearman     | 20   | Haas Ferrari                   |       0.07% |
| Gabriel Bortoleto  | 15   | Kick Sauber Ferrari            |       0.07% |
| Jack Doohan        | 14   | Alpine Renault                 |       0.07% |
| Pierre Gasly       | 9    | Alpine Renault                 |       0.07% |
| Isack Hadjar       | 11   | Racing Bulls Honda RBPT        |       0.07% |
| Nico Hulkenberg    | 17   | Kick Sauber Ferrari            |       0.07% |
| Liam Lawson        | 18   | Red Bull Racing Honda RBPT     |       0.07% |
| Esteban Ocon       | 19   | Haas Ferrari                   |       0.07% |
| Carlos Sainz       | 10   | Williams Mercedes              |       0.07% |
| Lance Stroll       | 13   | Aston Martin Aramco Mercedes   |       0.07% |
| Yuki Tsunoda       | 5    | Racing Bulls Honda RBPT        |       0.07% |



--- Predicting: 2025 Chinese Grand Prix ---
Calculating features...
Merging names from qualifying data...
Saved: predictions_GBC_Sprint/2025_R02_2025_Chinese_Grand_Prix_shanghai_predictions_GBC_Sprint.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Oscar Piastri      | 1    | McLaren Mercedes               |      25.34% |
| Max Verstappen     | 4    | Red Bull Racing Honda RBPT     |       3.22% |
| George Russell     | 2    | Mercedes                       |       2.41% |
| Lando Norris       | 3    | McLaren Mercedes               |       1.92% |
| Kimi Antonelli     | 8    | Mercedes                       |       0.08% |
| Lewis Hamilton     | 5    | Ferrari                        |       0.05% |
| Alexander Albon    | 10   | Williams Mercedes              |       0.05% |
| Fernando Alonso    | 13   | Aston Martin Aramco Mercedes   |       0.05% |
| Oliver Bearman     | 17   | Haas Ferrari                   |       0.05% |
| Gabriel Bortoleto  | 19   | Kick Sauber Ferrari            |       0.05% |
| Jack Doohan        | 18   | Alpine Renault                 |       0.05% |
| Pierre Gasly       | 16   | Alpine Renault                 |       0.05% |
| Isack Hadjar       | 7    | Racing Bulls Honda RBPT        |       0.05% |
| Nico Hulkenberg    | 12   | Kick Sauber Ferrari            |       0.05% |
| Liam Lawson        | 20   | Red Bull Racing Honda RBPT     |       0.05% |
| Esteban Ocon       | 11   | Haas Ferrari                   |       0.05% |
| Carlos Sainz       | 15   | Williams Mercedes              |       0.05% |
| Lance Stroll       | 14   | Aston Martin Aramco Mercedes   |       0.05% |
| Yuki Tsunoda       | 9    | Racing Bulls Honda RBPT        |       0.05% |
| Charles Leclerc    | 6    | Ferrari                        |       0.04% |



--- Predicting: 2025 Japanese Grand Prix ---
Calculating features...
Merging names from qualifying data...
Saved: predictions_GBC_Sprint/2025_R03_2025_Japanese_Grand_Prix_suzuka_predictions_GBC_Sprint.csv


...

| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| George Russell     | 1    | Mercedes                       |      29.59% |
| Max Verstappen     | 2    | Red Bull Racing Honda RBPT     |      12.65% |
| Oscar Piastri      | 3    | McLaren Mercedes               |       1.02% |
| Lando Norris       | 5    | McLaren Mercedes               |       0.35% |
| Kimi Antonelli     | 4    | Mercedes                       |       0.25% |
| Charles Leclerc    | 7    | Ferrari                        |       0.08% |
| Lewis Hamilton     | 6    | Ferrari                        |       0.08% |
| Alexander Albon    | 19   | Williams Mercedes              |       0.07% |
| Fernando Alonso    | 10   | Aston Martin Aramco Mercedes   |       0.07% |
| Oliver Bearman     | 9    | Haas Ferrari                   |       0.07% |
| Gabriel Bortoleto  | 14   | Kick Sauber Ferrari            |       0.07% |
| Franco Colapinto   | 16   | Alpine Renault                 |       0.07% |
| Pierre Gasly       | 20   | Alpine Renault                 |       0.07% |
| Isack Hadjar       | 8    | Racing Bulls Honda RBPT        |       0.07% |
| Nico Hulkenberg    | 11   | Kick Sauber Ferrari            |       0.07% |
| Liam Lawson        | 12   | Racing Bulls Honda RBPT        |       0.07% |
| Esteban Ocon       | 17   | Haas Ferrari                   |       0.07% |
| Carlos Sainz       | 18   | Williams Mercedes              |       0.07% |
| Lance Stroll       | 15   | Aston Martin Aramco Mercedes   |       0.07% |
| Yuki Tsunoda       | 13   | Red Bull Racing Honda RBPT     |       0.07% |



--- Predicting: 2025 United States Grand Prix ---
Calculating features...
Merging names from qualifying data...
Saved: predictions_GBC_Sprint/2025_R19_2025_United_States_Grand_Prix_americas_predictions_GBC_Sprint.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Max Verstappen     | 1    | Red Bull Racing Honda RBPT     |      23.05% |
| Lando Norris       | 2    | McLaren Mercedes               |       0.95% |
| Charles Leclerc    | 3    | Ferrari                        |       0.88% |
| George Russell     | 4    | Mercedes                       |       0.35% |
| Oscar Piastri      | 6    | McLaren Mercedes               |       0.13% |
| Lewis Hamilton     | 5    | Ferrari                        |       0.07% |
| Yuki Tsunoda       | 13   | Red Bull Racing Honda RBPT     |       0.04% |
| Alexander Albon    | 18   | Williams Mercedes              |       0.04% |
| Fernando Alonso    | 10   | Aston Martin Aramco Mercedes   |       0.04% |
| Kimi Antonelli     | 7    | Mercedes                       |       0.04% |
| Oliver Bearman     | 8    | Haas Ferrari                   |       0.04% |
| Gabriel Bortoleto  | 16   | Kick Sauber Ferrari            |       0.04% |
| Franco Colapinto   | 15   | Alpine Renault                 |       0.04% |
| Pierre Gasly       | 14   | Alpine Renault                 |       0.04% |
| Isack Hadjar       | 19   | Racing Bulls Honda RBPT        |       0.04% |
| Nico Hulkenberg    | 11   | Kick Sauber Ferrari            |       0.04% |
| Liam Lawson        | 12   | Racing Bulls Honda RBPT        |       0.04% |
| Esteban Ocon       | 17   | Haas Ferrari                   |       0.04% |
| Carlos Sainz       | 9    | Williams Mercedes              |       0.04% |
| Lance Stroll       | 20   | Aston Martin Aramco Mercedes   |       0.04% |



--- Predicting: 2025 Mexican Grand Prix ---
Calculating features...
Merging names from qualifying data...
Saved: predictions_GBC_Sprint/2025_R20_2025_Mexican_Grand_Prix_rodriguez_predictions_GBC_Sprint.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Max Verstappen     | 5    | Red Bull Racing Honda RBPT     |      52.52% |
| Lando Norris       | 1    | McLaren Mercedes               |      32.33% |
| Charles Leclerc    | 2    | Ferrari                        |       9.55% |
| Lewis Hamilton     | 3    | Ferrari                        |       1.57% |
| George Russell     | 4    | Mercedes                       |       1.45% |
| Oscar Piastri      | 8    | McLaren Mercedes               |       0.43% |
| Yuki Tsunoda       | 11   | Red Bull Racing Honda RBPT     |       0.16% |
| Alexander Albon    | 17   | Williams Mercedes              |       0.15% |
| Fernando Alonso    | 14   | Aston Martin Aramco Mercedes   |       0.15% |
| Kimi Antonelli     | 6    | Mercedes                       |       0.15% |
| Oliver Bearman     | 10   | Haas Ferrari                   |       0.15% |
| Gabriel Bortoleto  | 16   | Kick Sauber Ferrari            |       0.15% |
| Franco Colapinto   | 20   | Alpine Renault                 |       0.15% |
| Pierre Gasly       | 18   | Alpine Renault                 |       0.15% |
| Isack Hadjar       | 9    | Racing Bulls Honda RBPT        |       0.15% |
| Nico Hulkenberg    | 13   | Kick Sauber Ferrari            |       0.15% |
| Liam Lawson        | 15   | Racing Bulls Honda RBPT        |       0.15% |
| Esteban Ocon       | 12   | Haas Ferrari                   |       0.15% |
| Carlos Sainz       | 7    | Williams Mercedes              |       0.15% |
| Lance Stroll       | 19   | Aston Martin Aramco Mercedes   |       0.15% |
